From AI Engineering videos by Srivatsan Srinivasan

In [ ]:
!nvidia-smi

Tue Sep 15 03:25:11 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz


In [ ]:
!tar xf spark-3.0.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
!wget https://repo1.maven.org/maven2/ai/rapids/cudf/0.14/cudf-0.14-cuda10-1.jar
!wget https://repo1.maven.org/maven2/com/nvidia/xgboost4j_3.0/1.0.0-0.1.0/xgboost4j_3.0-1.0.0-0.1.0.jar
!wget https://repo1.maven.org/maven2/com/nvidia/xgboost4j-spark_3.0/1.0.0-0.1.0/xgboost4j-spark_3.0-1.0.0-0.1.0.jar
!wget http://insecure.repo1.maven.org/maven2/com/nvidia/rapids-4-spark_2.12/0.1.0/rapids-4-spark_2.12-0.1.0.jar

--2020-09-15 03:25:31--  https://repo1.maven.org/maven2/ai/rapids/cudf/0.14/cudf-0.14-cuda10-1.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 145993287 (139M) [application/java-archive]
Saving to: ‘cudf-0.14-cuda10-1.jar’

cudf-0.14-cuda10-1. 100%[===================>] 139.23M   179MB/s    in 0.8s    

2020-09-15 03:25:36 (179 MB/s) - ‘cudf-0.14-cuda10-1.jar’ saved [145993287/145993287]

--2020-09-15 03:25:36--  https://repo1.maven.org/maven2/com/nvidia/xgboost4j_3.0/1.0.0-0.1.0/xgboost4j_3.0-1.0.0-0.1.0.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 231556205 (221M) [application/java-archive]
Saving to: ‘xgboost4j_3.0-1.0.0-0.1.0.jar’

xgboo

In [ ]:
!ls
!pwd

cudf-0.14-cuda10-1.jar	       xgboost4j_3.0-1.0.0-0.1.0.jar
rapids-4-spark_2.12-0.1.0.jar  xgboost4j-spark_3.0-1.0.0-0.1.0.jar
sample_data
/content


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"

In [ ]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /content/cudf-0.14-cuda10-1.jar,/content/xgboost4j_3.0-1.0.0-0.1.0.jar,/content/xgboost4j-spark_3.0-1.0.0-0.1.0.jar,/content/rapids-4-spark_2.12-0.1.0.jar pyspark-shell'

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").config("spark.plugins", "com.nvidia.spark.SQLPlugin").config("spark.rapids.memory.gpu.pooling.enabled",False).getOrCreate()
spark.sparkContext.addPyFile('/content/xgboost4j-spark_3.0-1.0.0-0.1.0.jar')
spark.sparkContext.addPyFile('/content/rapids-4-spark_2.12-0.1.0.jar')

In [ ]:
reader = spark.read

In [ ]:
from ml.dmlc.xgboost4j.scala.spark import XGBoostClassificationModel, XGBoostClassifier

import numpy as np
import pandas as pd

In [ ]:
!wget https://raw.githubusercontent.com/srivatsan88/YouTubeLI/master/dataset/covtype_train.parquet
!wget https://raw.githubusercontent.com/srivatsan88/YouTubeLI/master/dataset/covtype_test.parquet

--2020-09-15 03:29:50--  https://raw.githubusercontent.com/srivatsan88/YouTubeLI/master/dataset/covtype_train.parquet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6827427 (6.5M) [application/octet-stream]
Saving to: ‘covtype_train.parquet’

covtype_train.parqu 100%[===================>]   6.51M  34.7MB/s    in 0.2s    

2020-09-15 03:29:51 (34.7 MB/s) - ‘covtype_train.parquet’ saved [6827427/6827427]

--2020-09-15 03:29:51--  https://raw.githubusercontent.com/srivatsan88/YouTubeLI/master/dataset/covtype_test.parquet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting 

In [ ]:
!ls -alrt

total 815020
-rw-r--r--  1 root root 145993287 Jun  9 13:17 cudf-0.14-cuda10-1.jar
-rw-r--r--  1 root root   6676958 Jun 20 02:24 rapids-4-spark_2.12-0.1.0.jar
-rw-r--r--  1 root root 231556205 Jun 26 03:20 xgboost4j_3.0-1.0.0-0.1.0.jar
-rw-r--r--  1 root root   2040779 Jun 26 03:21 xgboost4j-spark_3.0-1.0.0-0.1.0.jar
drwxr-xr-x 13 1000 1000      4096 Aug 28 08:10 spark-3.0.1-bin-hadoop2.7
-rw-r--r--  1 root root 219929956 Aug 28 09:25 spark-3.0.1-bin-hadoop2.7.tgz.1
-rw-r--r--  1 root root 219929956 Aug 28 09:25 spark-3.0.1-bin-hadoop2.7.tgz
drwxr-xr-x  1 root root      4096 Sep 11 16:28 sample_data
drwxr-xr-x  1 root root      4096 Sep 11 16:28 .config
drwxr-xr-x  1 root root      4096 Sep 15 03:21 ..
-rw-r--r--  1 root root   6827427 Sep 15 03:29 covtype_train.parquet
drwxr-xr-x  1 root root      4096 Sep 15 03:29 .
-rw-r--r--  1 root root   1591040 Sep 15 03:29 covtype_test.parquet


In [ ]:
import pyarrow.parquet as pq

In [ ]:
pq.read_table('covtype_train.parquet')

pyarrow.Table
Elevation: double
Aspect: double
Slope: double
Horizontal_Distance_To_Hydrology: double
Vertical_Distance_To_Hydrology: double
Horizontal_Distance_To_Roadways: double
Hillshade_9am: double
Hillshade_Noon: double
Hillshade_3pm: double
Horizontal_Distance_To_Fire_Points: double
Wilderness_Area1: double
Wilderness_Area2: double
Wilderness_Area3: double
Wilderness_Area4: double
Soil_Type1: double
Soil_Type2: double
Soil_Type3: double
Soil_Type4: double
Soil_Type5: double
Soil_Type6: double
Soil_Type7: double
Soil_Type8: double
Soil_Type9: double
Soil_Type10: double
Soil_Type11: double
Soil_Type12: double
Soil_Type13: double
Soil_Type14: double
Soil_Type15: double
Soil_Type16: double
Soil_Type17: double
Soil_Type18: double
Soil_Type19: double
Soil_Type20: double
Soil_Type21: double
Soil_Type22: double
Soil_Type23: double
Soil_Type24: double
Soil_Type25: double
Soil_Type26: double
Soil_Type27: double
Soil_Type28: double
Soil_Type29: double
Soil_Type30: double
Soil_Type31: doubl

In [ ]:
train_data = reader.format('parquet').load('covtype_train.parquet')
test_data = reader.format('parquet').load('covtype_test.parquet')

In [ ]:
train_data.show()

+---------+------+-----+--------------------------------+------------------------------+-------------------------------+-------------+--------------+-------------+----------------------------------+----------------+----------------+----------------+----------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------+
|Elevation|Aspect|Slope|Horizontal_Distance_To_Hydrology|Vertical_Distance_To_Hydrology|Horizontal_Distance_To_Roadways|Hillshade_9am|Hillshade_Noon|Hillshade_3pm|Horizontal_Distance_To_Fire_Points|Wilderness_Area1|Wilderness_Area2|Wilderness_Area3|Wilder

In [ ]:
train_data.schema

StructType(List(StructField(Elevation,DoubleType,true),StructField(Aspect,DoubleType,true),StructField(Slope,DoubleType,true),StructField(Horizontal_Distance_To_Hydrology,DoubleType,true),StructField(Vertical_Distance_To_Hydrology,DoubleType,true),StructField(Horizontal_Distance_To_Roadways,DoubleType,true),StructField(Hillshade_9am,DoubleType,true),StructField(Hillshade_Noon,DoubleType,true),StructField(Hillshade_3pm,DoubleType,true),StructField(Horizontal_Distance_To_Fire_Points,DoubleType,true),StructField(Wilderness_Area1,DoubleType,true),StructField(Wilderness_Area2,DoubleType,true),StructField(Wilderness_Area3,DoubleType,true),StructField(Wilderness_Area4,DoubleType,true),StructField(Soil_Type1,DoubleType,true),StructField(Soil_Type2,DoubleType,true),StructField(Soil_Type3,DoubleType,true),StructField(Soil_Type4,DoubleType,true),StructField(Soil_Type5,DoubleType,true),StructField(Soil_Type6,DoubleType,true),StructField(Soil_Type7,DoubleType,true),StructField(Soil_Type8,DoubleType

In [ ]:
pq_file=pq.read_table('covtype_train.parquet')

In [ ]:
pq_file.column_names

['Elevation',
 'Aspect',
 'Slope',
 'Horizontal_Distance_To_Hydrology',
 'Vertical_Distance_To_Hydrology',
 'Horizontal_Distance_To_Roadways',
 'Hillshade_9am',
 'Hillshade_Noon',
 'Hillshade_3pm',
 'Horizontal_Distance_To_Fire_Points',
 'Wilderness_Area1',
 'Wilderness_Area2',
 'Wilderness_Area3',
 'Wilderness_Area4',
 'Soil_Type1',
 'Soil_Type2',
 'Soil_Type3',
 'Soil_Type4',
 'Soil_Type5',
 'Soil_Type6',
 'Soil_Type7',
 'Soil_Type8',
 'Soil_Type9',
 'Soil_Type10',
 'Soil_Type11',
 'Soil_Type12',
 'Soil_Type13',
 'Soil_Type14',
 'Soil_Type15',
 'Soil_Type16',
 'Soil_Type17',
 'Soil_Type18',
 'Soil_Type19',
 'Soil_Type20',
 'Soil_Type21',
 'Soil_Type22',
 'Soil_Type23',
 'Soil_Type24',
 'Soil_Type25',
 'Soil_Type26',
 'Soil_Type27',
 'Soil_Type28',
 'Soil_Type29',
 'Soil_Type30',
 'Soil_Type31',
 'Soil_Type32',
 'Soil_Type33',
 'Soil_Type34',
 'Soil_Type35',
 'Soil_Type36',
 'Soil_Type37',
 'Soil_Type38',
 'Soil_Type39',
 'Soil_Type40',
 'target']

In [ ]:
label="target"
features = [ x for x in pq_file.column_names if x != label ]

In [ ]:
features

['Elevation',
 'Aspect',
 'Slope',
 'Horizontal_Distance_To_Hydrology',
 'Vertical_Distance_To_Hydrology',
 'Horizontal_Distance_To_Roadways',
 'Hillshade_9am',
 'Hillshade_Noon',
 'Hillshade_3pm',
 'Horizontal_Distance_To_Fire_Points',
 'Wilderness_Area1',
 'Wilderness_Area2',
 'Wilderness_Area3',
 'Wilderness_Area4',
 'Soil_Type1',
 'Soil_Type2',
 'Soil_Type3',
 'Soil_Type4',
 'Soil_Type5',
 'Soil_Type6',
 'Soil_Type7',
 'Soil_Type8',
 'Soil_Type9',
 'Soil_Type10',
 'Soil_Type11',
 'Soil_Type12',
 'Soil_Type13',
 'Soil_Type14',
 'Soil_Type15',
 'Soil_Type16',
 'Soil_Type17',
 'Soil_Type18',
 'Soil_Type19',
 'Soil_Type20',
 'Soil_Type21',
 'Soil_Type22',
 'Soil_Type23',
 'Soil_Type24',
 'Soil_Type25',
 'Soil_Type26',
 'Soil_Type27',
 'Soil_Type28',
 'Soil_Type29',
 'Soil_Type30',
 'Soil_Type31',
 'Soil_Type32',
 'Soil_Type33',
 'Soil_Type34',
 'Soil_Type35',
 'Soil_Type36',
 'Soil_Type37',
 'Soil_Type38',
 'Soil_Type39',
 'Soil_Type40']

In [ ]:
import time
params = { 
    'eta': 0.1,
    'gamma': 0.1,
    'missing': 0.0,
    'treeMethod': 'gpu_hist',
    'maxDepth': 6, 
    'growPolicy': 'depthwise',
    'lambda_': 1.0,
    'subsample': 1.0,
    'numRound': 1000,
    'numWorkers': 1,
    'verbosity': 2
}
classifier = XGBoostClassifier(**params).setLabelCol(label).setFeaturesCols(features)

In [ ]:
start_time = time.time()

model=classifier.fit(train_data)

print("GPU Training Time: %s seconds" % (str(time.time() - start_time)))

GPU Training Time: 14.906355857849121 seconds


In [ ]:
!nvidia-smi

Tue Sep 15 03:31:11 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    41W / 250W |   1339MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
model.write().overwrite().save('/content/model/')

In [ ]:
!ls

covtype_test.parquet	       spark-3.0.1-bin-hadoop2.7
covtype_train.parquet	       spark-3.0.1-bin-hadoop2.7.tgz
cudf-0.14-cuda10-1.jar	       spark-3.0.1-bin-hadoop2.7.tgz.1
model			       xgboost4j_3.0-1.0.0-0.1.0.jar
rapids-4-spark_2.12-0.1.0.jar  xgboost4j-spark_3.0-1.0.0-0.1.0.jar
sample_data


In [ ]:
loaded_model = XGBoostClassificationModel().load('/content/model/')

In [ ]:
result=loaded_model.transform(test_data)

In [ ]:
result.select(label, 'rawPrediction', 'probability', 'prediction').show(5)

+------+--------------------+--------------------+----------+
|target|       rawPrediction|         probability|prediction|
+------+--------------------+--------------------+----------+
|     1|[-0.7645629644393...|[0.23543703556060...|       1.0|
|     4|[-3.3206388950347...|[-2.3206388950347...|       1.0|
|     4|[-3.2919535636901...|[-2.2919535636901...|       1.0|
|     4|[-3.3060050010681...|[-2.3060050010681...|       1.0|
|     4|[-2.9611465930938...|[-1.9611465930938...|       1.0|
+------+--------------------+--------------------+----------+
only showing top 5 rows



In [ ]:
MulticlassClassificationEvaluator().setLabelCol(label).evaluate(result)